In [ ]:
import pandas as pd
import bamboolib as bam
import datetime
import pyarrow as pa
import pyarrow.parquet as pq

Először beolvassuk a 2020-as adatokat hónaponként és egy listába tesszük őket.

In [ ]:
from calendar import monthrange
filtered2020monthlist=[]
for i in range(1,13):
    year = 2020
    month=i

    monthly_data = pd.read_parquet(
    path=f'{year}/{year}-{month:02d}.parquet',
    columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID',
             'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
    )

    filter_columns = ['fare_amount', 'tip_amount', 'total_amount']
    percentile = 0.9999

    filter_string = ' & '.join(
    (
        f'("{year}, {month}, 1" <= tpep_pickup_datetime)',
        f'(tpep_pickup_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
        f'("{year}, {month}, 1" <= tpep_dropoff_datetime)',
        f'(tpep_dropoff_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
        '(tpep_pickup_datetime <= tpep_dropoff_datetime)',
        '(1 <= passenger_count) & (passenger_count <= 6)',
        '(0 < trip_distance) & (trip_distance <= 330)',
        '(1 <= PULocationID) & (PULocationID <= 263)',
        '(1 <= DOLocationID) & (DOLocationID <= 263)',
        '(2.5 <= fare_amount)',
        '(extra in (0, 0.5, 1, 2.5, 3, 3.5))',
        '(0 <= tip_amount)',
        '(fare_amount <= total_amount)',
        *(f'({column} <= {column}.quantile({percentile}))' for column in filter_columns)
    )
)
    filtered2020monthlist.append(monthly_data.sample(n=len(monthly_data)//7).query(filter_string))
   # if len(monthly_data)>500000:
    #    filtered2020monthlist.append(monthly_data.sample(n=500000).query(filter_string))
   # else:
    #    filtered2020monthlist.append(monthly_data.query(filter_string))
    
    #filtered2020monthlist.append(monthly_data.query(filter_string))
    #filtered2020jul = monthly_data.query(filter_string)

Beolvassuk a 2020-as adatokat a 2019-esekhez hasonlóan.

In [ ]:
from calendar import monthrange
filtered2019monthlist=[]
for i in range(1,13):
    year = 2019
    month=i

    monthly_data = pd.read_parquet(
    path=f'{year}/{year}-{month:02d}.parquet',
    columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID',
             'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
    )

    filter_columns = ['fare_amount','tip_amount', 'total_amount']
    percentile = 0.9999

    filter_string = ' & '.join(
    (
        f'("{year}, {month}, 1" <= tpep_pickup_datetime)',
        f'(tpep_pickup_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
        f'("{year}, {month}, 1" <= tpep_dropoff_datetime)',
        f'(tpep_dropoff_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
        '(tpep_pickup_datetime <= tpep_dropoff_datetime)',
        '(1 <= passenger_count) & (passenger_count <= 6)',
        '(0 < trip_distance) & (trip_distance <= 330)',
        '(1 <= PULocationID) & (PULocationID <= 263)',
        '(1 <= DOLocationID) & (DOLocationID <= 263)',
        '(2.5 <= fare_amount)',
        '(extra in (0, 0.5, 1,2.5,3,3.5))',
        '(0 <= tip_amount)',
        '(fare_amount <= total_amount)',
        *(f'({column} <= {column}.quantile({percentile}))' for column in filter_columns)
        )
    )
    
    filtered2019monthlist.append(monthly_data.sample(n=len(monthly_data)//7).query(filter_string))
   # if len(monthly_data)>500000:
    #    filtered2019monthlist.append(monthly_data.sample(n=500000).query(filter_string))
   # else:
    #    filtered2019monthlist.append(monthly_data.query(filter_string))
    
    #filtered2019monthlist.append(monthly_data.query(filter_string))
    #filtered2020jul = monthly_data.query(filter_string)

Összetesszük a hónapos adatokat évesekre. Nagyon sok adat van, amit a jupyter notebookban nem tudunk kezelni,
ezért mindegyik hónapból maximum 500000 mintát veszünk. Ekkora minta még reprezentatív, de már egyszerűbb kezelni.

In [ ]:
ossz2020=pd.concat(filtered2020monthlist)
ossz2020.to_parquet('ossz/osszsamp2020.parquet')

In [ ]:
ossz2019=pd.concat(filtered2019monthlist)
ossz2019.to_parquet('ossz/osszsamp2019.parquet')

Összetesszük a két éves adathalmazt, hogy bamboolibbel elemezni tudjuk.

In [ ]:

osszfiltered2019=pd.read_parquet('ossz/osszsamp2019.parquet')
osszfiltered2019['year']="2019"
osszfiltered2019['hetvege']=osszfiltered2019['tpep_pickup_datetime'].dt.weekday < 5
osszfiltered2020=pd.read_parquet('ossz/osszsamp2020.parquet')
osszfiltered2020['year']="2020"
osszfiltered2020['hetvege']=osszfiltered2019['tpep_pickup_datetime'].dt.weekday < 5
osszfiltered=pd.concat([osszfiltered2019,osszfiltered2020])

A borravló vizsgálat szempontjából csak a kártyás fizetésre van szükségünk, mivel csak azoknál mértek borravalót.

In [ ]:
onlycarddata=osszfiltered[osszfiltered.payment_type==1]

In [ ]:
onlycarddata

In [ ]:
osszfiltered2019

In [ ]:
osszfiltered

Ezen a ploton azt vizsgáljuk meg hogy hogyan függ az utazás távolsága az extra költségektől. Extra költség típusok: 0.50 overnight charge(8pm-6am), 1 rush hour charge(4pm-8pm), New York State Congestion Surcharge(Ha manhattan 96th útja déli részét érinti az út) of 2.50. Az útóbbit nem tartalmazza az adatszolgáltatók szerint, de az adatokból úgy tűnik, hogy tartalmazza.

A diagrammon az látszik hogy éjszaka hosszabb távon igényelték a taxikat mindkét évben, mint normál időpontokban. 2019-ben rush órákban rövidebb távot utaztak, mint normál, nem extra költésges időkben, még ez 2020-ben pont fordítva volt. Itt pont az ellentettjére számítottunk, mert gondoltuk, hogy 2020-ban a munkahelyüktől távolabb élők nagyobb számban dolgoztak otthonról, mint a közelebb élők. A 96th utcát érintő taxizások hossza kicsivel rövidebb mint az azt nem érintőké.

In [ ]:
import plotly.express as px
fig = px.box(osszfiltered.sample(n=300000, replace=False).sort_index(), x='extra', y='trip_distance', color='year')
fig.update_yaxes(type='log', tickformat='.1e')
fig

Ezen a ploton a borravaló átlagos mennyiségét vizsgáljuk az extra költésgek szempontjából. Ehhez a plothoz csak a kártyával fizetők adatait használtuk, mert csak náluk jelenítették meg a borravalót.

A ploton látható, hogy mind a rush órákban mind az éjszakai órákban a borravaló átlagos mennyisége nagyobb a normál órákhoz képest. A rush órákban és éjszakai órákban körülbelül ugyanannyi az átlagos borravaló. A 96th utcát érintő taxizásoknál nem volt nagy eltérés az azt nem érintőkhöz képest.

In [ ]:
import plotly.express as px
fig = px.box(onlycarddata.sample(n=150000, replace=False).sort_index(), x='extra', y='tip_amount', color='year')
fig.update_yaxes(type='log', tickformat='.1e')
fig

Ezen a ploton a borravalót és a út hosszát hasonlítjuk össze.Sok egy sorban levő adat van, ezek a kerek mennyiségű borravalók miatt vannak. A 0 sorban azok az adatok vannak ahol nem adtak borravalót.

In [ ]:
import plotly.express as px
fig = px.strip(onlycarddata.sample(n=20000, replace=False, random_state=123).sort_index(), x='trip_distance', y='tip_amount', color='year')
fig

In [ ]:
import plotly.express as px
fig = px.box(osszfiltered.sample(n=500000, replace=False).sort_index(), x='extra', y='total_amount', color='year')
fig.update_yaxes(type='log', tickformat='.1e')
fig

In [ ]:
import plotly.express as px
fig = px.box(osszfiltered.sample(n=100000, replace=False, random_state=95).sort_index(), x='extra', y='total_amount', color='year')
fig.update_yaxes(type='log', tickformat='.1e')
fig

Ezen a ploton az átlagos fare amountot vizsgáljuk évenként. A fare amount a taxi óra által számított ár. Ebben nincsenek az extra költségek benne.
2019-ben magasabb volt az átlagos (vitelár?) fare amount, mint 2020-ban.

In [ ]:
import plotly.express as px
fig = px.histogram(osszfiltered.sample(n=100000, replace=False).sort_index(), x='year', y='fare_amount', histfunc='avg', color_discrete_sequence=px.colors.qualitative.Dark2, color='year')
fig

A fare amounthoz hasonlóan a távolság is átlagosan magasabb volt 2019-ben mint 2020-ban. Ez a kettő erősen összefügg mivel a fare amount jelentős része a távolságdíjból származik

In [ ]:
import plotly.express as px
fig = px.histogram(osszfiltered.sample(n=100000, replace=False).sort_index(), x='year', y='trip_distance', histfunc='avg', color_discrete_sequence=px.colors.qualitative.Dark2, color='year')
fig

Itt a idő során az utazás hosszát jelenítjük meg

In [ ]:
import plotly.express as px
fig = px.histogram(osszfiltered.sample(n=400000, replace=False).sort_index(), x='tpep_pickup_datetime', y='trip_distance', histfunc='sum', nbins=200)
fig

In [ ]:
import plotly.express as px
fig = px.histogram(osszfiltered.sample(n=400000, replace=False).sort_index(), x='tpep_pickup_datetime', y='total_amount', histfunc='sum', nbins=120)
fig

In [ ]:
import plotly.express as px
fig = px.histogram([osszfiltered2019.sample(n=100000, replace=False).sort_index(),osszfiltered2020.sample(n=100000, replace=False).sort_index()], x='hetvege', y='total_amount', histfunc='avg')
fig